In [1]:
import helpy
import numpy as np
import cv2
import matplotlib.pyplot as plt
from skimage.transform import resize

In [2]:
class build_training_data:
    def __init__(self, image_file, frame_data, training_dict = None, number_test = None,\
                 training_shape = 32, upscale_size = 4, start_particle = 0, \
                 image_size = 1000, save_name = "2inch_square_training_data.npy"):
        self.image_file = image_file
        self.frame_data = frame_data
        self.training_shape = training_shape
        self.centralize = training_shape // 2
        self.upscale_size = upscale_size
        if number_test:
            self.number_test = number_test
        else:
            self.number_test = len(frame_data)
        self.start_particle = start_particle
        self.image_size = image_size
        if training_dict is None:
            self.training_dict = {
                "training_class": list(),
                "training_regress": list(),
                "training_x": list()
            }
        elif isinstance(training_dict, str):
            self.training_dict = np.load(training_dict).item()
            self.save_name = training_dict
        else:
            self.training_dict = training_dict
            
    def prep_image(self, width = 2):
        im = plt.imread(self.image_file).astype(float)
        s = width*im.std()
        m = im.mean()
        im -= m - s
        im /= 2 * s
        np.clip(im,0,1,out=im)
        return im
    
    def necessary_data(self):
        pos = self.frame_data['xy'] - np.array([self.centralize, self.centralize])
        pos_start = pos.astype(int)
        pos_cen = (self.frame_data['xy'] - pos_start)*self.upscale_size
        pos_arrow = pos_cen + self.upscale_size * 15.0 * np.array([np.cos(frame_data['o']), np.sin(frame_data['o'])]).T
        pos_arrow = pos_arrow.astype(np.float32)
        pos_cen = pos_cen.astype(np.float32)
        return pos_start, pos_cen, pos_arrow
    
    def interactive_mode(self):
        cv2.namedWindow("Build", cv2.WINDOW_NORMAL)
        cv2.resizeWindow("Build", self.image_size, self.image_size)
        pi2 = np.pi * 2
        img = self.prep_image()
        pos_start, pos_cen, pos_arrow = self.necessary_data()
        for frame, (x_start,y_start),arr_cen, arr_pos in zip(self.frame_data['o'][self.start_particle:self.number_test],\
                                                             pos_start[self.start_particle:self.number_test],\
                                                             pos_cen[self.start_particle:self.number_test],\
                                                             pos_arrow[self.start_particle:self.number_test]):
            temp_img = img[x_start:x_start+self.training_shape, y_start:y_start + self.training_shape]
            resize_img = resize(temp_img, (self.training_shape*self.upscale_size, self.training_shape*self.upscale_size),\
                               mode = 'reflect', anti_aliasing = True)
            if np.isnan(arr_pos[0]):
                cv2.imshow("Build", resize_img)
            else:
                arrow_plot = cv2.arrowedLine(resize_img, (arr_cen[1],arr_cen[0]),(arr_pos[1], arr_pos[0]),(255,0,0),2)
                cv2.imshow('Build', resize_img)
            print("detect or is: {}".format(frame))
            press = cv2.waitKey(0)
            # everything is good, store current information into training sets
            if press == 49:
                self.training_dict["training_x"].append(temp_img)
                self.training_dict["training_regress"].append(frame)
                self.training_dict["training_class"].append(1)
            # enter plot mode
            elif press == 50: 
                points_store = []
                button_down = False
                cv2.imshow("Build", resize_img)
                def orientation_click(event, x, y, flags, param):
                    global button_down
                    if event == cv2.EVENT_LBUTTONUP and button_down:
                        button_down = False
                        points_store.append((x,y))
                        cv2.arrowedLine(resize_img, points_store[0], points_store[1], (255,0,0),1)
                        cv2.imshow("Build", resize_img)
                        
                    elif event == cv2.EVENT_MOUSEMOVE and button_down:
                        button_down = True
                        image = resize_img.copy()
                        cv2.arrowedLine(image, points_store[0], (x,y), (255,0,0),1)
                        cv2.imshow("Build", image)

                    elif event == cv2.EVENT_LBUTTONDOWN and len(points_store) < 2:
                        button_down = True
                        points_store.insert(0, (x,y))
                cv2.setMouseCallback('Build',orientation_click, points_store)
                enter = cv2.waitKey(0)
                orientation = np.arctan2(points_store[1][0]-points_store[0][0], points_store[1][1]-points_store[0][1])
                orientation = (orientation + pi2)%pi2
                print("Click orientation is: {}".format(orientation))
                if enter == 49:
                    self.training_dict["training_x"].append(temp_img)
                    self.training_dict["training_regress"].append(orientation)
                    self.training_dict["training_class"].append(1)
            # wrong detected particles goes to mode 3, no orientation detected give it class 1
            elif press == 51:
                self.training_dict["training_x"].append(temp_img)
                self.training_dict["training_regress"].append(np.nan)
                self.training_dict["training_class"].append(0)
            elif press == 27:
                break
        cv2.destroyAllWindows()
        for i in range (1,5):
            cv2.waitKey(1)
        return
    
    
    def get_training_dict(self):
        self.interactive_mode()
        np.save(self.save_name, self.training_dict)

In [3]:
im_file='/home/zhejun/Result/20181203/result_03041.tif'

In [4]:
data =helpy.load_data('/home/zhejun/Result/20181203/result')
data['o'] = (data['o'] + np.pi)%(2*np.pi)

In [5]:
frame_data = data[data['f']==3040]

In [7]:
class1 = build_training_data(im_file, frame_data, training_dict= "lab_training_dict.npy",start_particle=1)
class1.get_training_dict()

detect or is: 2.364772319793701
detect or is: 3.904386281967163
Click orientation is: 3.9269908169872414
detect or is: nan
Click orientation is: 2.312743594800814
detect or is: 3.9991743564605713
Click orientation is: 4.068887871591405
detect or is: 3.8043620586395264
detect or is: nan
Click orientation is: 2.5702551737561663
detect or is: nan
Click orientation is: 3.7274944198919253
detect or is: nan
detect or is: 4.096266746520996
Click orientation is: 4.221066777817443
detect or is: 3.6196446418762207
Click orientation is: 3.6635931844417406
detect or is: 2.6653599739074707
Click orientation is: 2.6476512846702125
detect or is: nan
Click orientation is: 3.6509829797908013
detect or is: nan
detect or is: 4.009367942810059
Click orientation is: 3.9217552627308336
detect or is: 2.2842659950256348
detect or is: 4.002652168273926
detect or is: 3.8416388034820557
Click orientation is: 3.8732790876840086
detect or is: 2.718991756439209
detect or is: 4.090033531188965
Click orientation is: 

Click orientation is: 4.124386376837123
detect or is: 0.16872453689575195
detect or is: 2.953185558319092
Click orientation is: 2.997438266476639
detect or is: 2.928950786590576
Click orientation is: 2.8809902618424523
detect or is: 3.349512815475464
Click orientation is: 3.2689003954606473
detect or is: 0.6435031890869141
Click orientation is: 0.668906702702798
detect or is: 1.5904316902160645
detect or is: nan
Click orientation is: 4.670746401286101
detect or is: 4.598845481872559
Click orientation is: 4.516583244314597
detect or is: 3.0223422050476074
Click orientation is: 2.972491427588416
detect or is: 6.098687648773193
Click orientation is: 6.124530044993185
detect or is: 2.1592135429382324
detect or is: 3.2246036529541016
Click orientation is: 3.313783468112732
detect or is: 5.957144260406494
Click orientation is: 5.980756019638699
detect or is: 6.179381370544434
Click orientation is: 6.234835920527684
detect or is: 1.7704596519470215
Click orientation is: 1.8138917529767404
det

detect or is: 3.685140371322632
Click orientation is: 3.597941122146902
detect or is: 2.3902649879455566
Click orientation is: 2.3729526977745827
detect or is: 2.7040553092956543
Click orientation is: 2.69580028972854
detect or is: 1.1411452293395996
Click orientation is: 1.1140690225443315
detect or is: nan
Click orientation is: 5.775348795541162
detect or is: 5.1505889892578125
Click orientation is: 5.230589915328554
detect or is: 4.150033473968506
detect or is: 1.6477751731872559
Click orientation is: 1.6110970750986784
detect or is: 2.628225803375244
detect or is: 0.6433477401733398
detect or is: 3.704195022583008
detect or is: 0.5772099494934082
Click orientation is: 0.6299067801288531
detect or is: 0.14528417587280273
detect or is: 4.397584915161133
detect or is: 3.7797374725341797
Click orientation is: 3.680698094169949
detect or is: 1.4869322776794434
detect or is: 6.1127119064331055
detect or is: 4.114260673522949
detect or is: nan
Click orientation is: 0.9890566893749968
dete

detect or is: 0.7257781028747559
detect or is: 5.772843837738037
detect or is: 2.275825023651123
Click orientation is: 2.340153085545701
detect or is: 2.9233946800231934
detect or is: 5.60343074798584
detect or is: 2.756364345550537
Click orientation is: 2.761086276477428
detect or is: 3.2923011779785156
detect or is: 0.18050289154052734
detect or is: 2.8542113304138184
detect or is: 1.1694512367248535
detect or is: 3.511643886566162
detect or is: 4.95854377746582
detect or is: 0.5799422264099121
Click orientation is: 0.6254850402392291
detect or is: 1.2366890907287598
detect or is: 3.1766517162323
detect or is: 1.8354554176330566
Click orientation is: 1.8289074431780588
detect or is: 2.356870174407959
Click orientation is: 2.3065116524683873
detect or is: 5.619976997375488
Click orientation is: 5.669270578088238
detect or is: 5.863388538360596
detect or is: 2.9586520195007324
detect or is: 5.682216644287109
detect or is: 5.024964809417725
detect or is: 3.894090175628662
Click orientat

Click orientation is: 5.508775712464871
detect or is: 5.950547218322754
detect or is: 4.894582271575928
detect or is: 5.97508430480957
detect or is: 1.4733262062072754
Click orientation is: 1.4150433460436043
detect or is: 4.385223388671875
Click orientation is: 4.433007070719862
detect or is: 4.897863388061523
detect or is: 0.36167001724243164
detect or is: 1.342785358428955
detect or is: 1.010016918182373
detect or is: 0.36547136306762695
Click orientation is: 0.3419498269777099
detect or is: nan
Click orientation is: 0.9937581961251469
detect or is: 0.7704634666442871
detect or is: 5.2267069816589355
detect or is: 5.873839378356934
detect or is: 5.361549377441406
detect or is: 5.965274333953857
detect or is: 2.5121006965637207
detect or is: 4.675295352935791
detect or is: 4.524960994720459
detect or is: 1.8306317329406738
Click orientation is: 1.7307094499531157
detect or is: 6.107287406921387
detect or is: 1.865644931793213
Click orientation is: 1.859976949179412
detect or is: 0.01

Click orientation is: 0.22131444234779174
detect or is: 1.2785582542419434
Click orientation is: 1.2341215074081697
detect or is: 5.972565174102783
Click orientation is: 6.026668694535263
detect or is: 4.833550453186035
detect or is: 2.392341136932373
detect or is: nan
Click orientation is: 1.1987285679367758
detect or is: 4.476810932159424
Click orientation is: 4.519688221722304
detect or is: 1.11604642868042
Click orientation is: 1.1789429189731653
detect or is: 4.908802509307861
Click orientation is: 4.908194716454783
detect or is: 6.107134819030762
Click orientation is: 6.030631031401526
detect or is: 5.383929252624512
Click orientation is: 5.3774916668541834
detect or is: 5.194555282592773
Click orientation is: 5.190220035376805
detect or is: 0.15566062927246094
Click orientation is: 0.19739555984988044
detect or is: 5.932971477508545
Click orientation is: 5.863305349974034
detect or is: 0.5690531730651855
detect or is: 5.753342628479004
Click orientation is: 5.616787437164918
det

In [8]:
d = class1.training_dict

In [10]:
len(d['training_x'])

2550

In [109]:
len(class1.training_dict['training_class'])

788

In [38]:
im_file='/home/zhejun/Result/20181203/result_03041.tif'

In [39]:
data =helpy.load_data('/home/zhejun/Result/20181203/result')
data['o'] = (data['o'] + np.pi)%(2*np.pi)


In [40]:
frame_data = data[data['f']==3040]

In [30]:
len(frame_data)

887

In [25]:
class1 = build_training_data(im_file, frame_data)

In [87]:
class T:
    a = 3
    def __init__(self):
        self.b = []
    
    def test(self):
        a = 4
        for i in range(10):
            self.b.append(i)
        print self.b
        print a
        print self.a

In [94]:
m = {
    "a": 3,
    "b": 4
}

In [89]:
tt.test()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
4
3


In [90]:
tt.b

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [21]:
b = list(a)

In [22]:
b.append(np.random.rand(3,3))

In [23]:
b

[array([[0.12007083, 0.31236005, 0.39983972],
        [0.20795855, 0.26741788, 0.79793932],
        [0.24240908, 0.84201298, 0.79312201]]),
 array([[0.23329187, 0.93620566, 0.79562605],
        [0.22787315, 0.38248369, 0.96826021],
        [0.21167955, 0.46250039, 0.86538129]]),
 array([[0.78900815, 0.37224248, 0.28179681],
        [0.18699984, 0.07279788, 0.38413059],
        [0.25596358, 0.32216125, 0.90411345]]),
 array([[0.81235322, 0.89896871, 0.92387773],
        [0.39337494, 0.21870837, 0.73681468],
        [0.37939043, 0.29513041, 0.34079612]])]